# preprocess

> Preprocess text data.

In [ ]:
#| default_exp preprocess

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from sklearn.base import BaseEstimator, TransformerMixin
import spacy
from textplumber.store import TextFeatureStore
import textstat

In [ ]:
#| export
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    import spacy.cli
    spacy.cli.download("en_core_web_sm")


In [ ]:
#| export
class SpacyPreprocessor(BaseEstimator, TransformerMixin):
	""" A Sci-kit Learn pipeline component to preprocess text using spaCy, 
		the pipeline component receives and returns texts, but prepares tokens, pos, and text statistics 
		as input to other compatible classes in a pipeline. """
	def __init__(self, 
				feature_store: TextFeatureStore, # the feature store to use
				model_name:str = 'en_core_web_sm', # the spaCy model to use
				disable: list[str] = ['parser', 'ner'], # the spaCy components to disable
				enable: list[str] = ['sentencizer'], # the spaCy components to enable
				batch_size:int = 500, # the batch size to use
				n_process:int = 1 # the number of processes to use
				 ):
		self.model_name = model_name
		self.disable = disable
		self.enable = enable
		self.batch_size = batch_size
		self.n_process = n_process
		self.nlp = spacy.load(self.model_name, disable=self.disable)
		for component in self.enable:
			self.nlp.add_pipe(component)

		self.feature_store = feature_store

	def _iterator(self, 
			   X:list # the texts to iterate over
			   ):
		""" Iterator to yield texts one by one. """
		for text in X:
			if text is None:
				yield ''
			else:
				yield str(text)

	def _fit_textstats(self, 
					doc:spacy.tokens.doc.Doc, # the spaCy document to fit text statistics for 
					tokens:list # the tokens to fit text statistics for
					) -> list: # the text statistics for the document
		""" Fit textstats for a document."""
		textstats = []
		textstats.append(textstat.monosyllabcount(doc.text)) # monosyllable relfreq
		textstats.append(textstat.polysyllabcount(doc.text)) # polysyllable relfreq
		textstats.append(len(tokens)) # token count
		textstats.append(len(list(doc.sents))) # sentence count
		textstats.append(len(set([token.lower() for token in tokens]))) # unique_tokens_count
		if len(list(doc.sents)) > 0:
			textstats.append(len(tokens)/len(list(doc.sents))) # average sentence length
		else:
			textstats.append(0) # average sentence length is zero when no sentences
		return textstats

	def _spacy_tokenize(self, 
					 doc:spacy.tokens.doc.Doc # the spaCy document to tokenize
					 ) -> tuple[list[str], list[str]]: # returns lists of tokens and part of speech tags
		return [token.text for token in doc if not token.is_space], [token.pos_ for token in doc if not token.is_space]
	
	def fit(self, X, y=None):
		""" Fit is implemented, but does nothing. """
		return self
	
	def transform(self, X):
		""" Preprocess the texts using spaCy and populate the feature store ready for use later in a pipeline. """

		tokens = self.feature_store.get_tokens_from_texts(X)
		if any(x is None for x in tokens):
			for doc in self.nlp.pipe(self._iterator(X), batch_size=self.batch_size, n_process=self.n_process):
				tokens, pos = self._spacy_tokenize(doc)
				textstats = self._fit_textstats(doc, tokens)
				self.feature_store.update(doc.text, tokens, pos, textstats)
		else:
			# all the tokens are already in the feature store so no need to reprocess
			pass
		return X


TODO: add an example.

In [ ]:
#| hide
import os
feature_store = TextFeatureStore('../test-data/test_preprocess')
spacy_preprocessor = SpacyPreprocessor(feature_store=feature_store)

spacy_preprocessor.fit([''])
spacy_preprocessor.transform([''])
assert spacy_preprocessor.feature_store.get('')['tokens'] == []

spacy_preprocessor.fit(['Hello, world!'])
spacy_preprocessor.transform(['Hello, world!'])
assert spacy_preprocessor.feature_store.get('Hello, world!')['tokens'] == ['Hello', ',', 'world', '!']
assert spacy_preprocessor.feature_store.get('Hello, world!')['pos'] == ['INTJ', 'PUNCT', 'NOUN', 'PUNCT']
spacy_preprocessor.feature_store.get('Hello, world!')['textstats']
assert spacy_preprocessor.feature_store.get('Hello, world!')['textstats'] == [1, 0, 4, 1, 4, 4.0]
# checking that the feature store has been populated ...
assert spacy_preprocessor.feature_store.get('Hello, world!') == {'tokens': ['Hello', ',', 'world', '!'], 'pos': ['INTJ', 'PUNCT', 'NOUN', 'PUNCT'], 'textstats': [1, 0, 4, 1, 4, 4.0], 'embeddings': None, 'lexicons': None}
del feature_store
os.remove('../test-data/test_preprocess')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()